# Assignment 2

Data Wrangling II
Create an “Academic performance” dataset of students and perform the
following operations using Python. 

1. Scan all variables for missing values and inconsistencies. 
If there are missing values and/or inconsistencies, use any of the suitable techniques to deal with them.

2. Scan all numeric variables for outliers. 
If there are outliers, use any of the suitable techniques to deal with them.

3. Apply data transformations on at least one of the variables. 
The purpose of this transformation should be one of the following reasons: 
- to change the scale for better understanding of the variable
- to convert a non-linear relation into a linear one
- to decrease the skewness and convert the distribution into a normal distribution.

Reason and document your approach properly.